In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("../results/enriched_routes/beynes_routes_enriched.csv", quotechar='"')



In [ ]:

#  cechy do normalizacji
continuous_features = [
    'free_flow_time', 'total_length', 'mean_speed', 'speed_std', 'speed_range',
    'lane_changes_per_km', 'priority_changes_per_km', 'yield_priority_changes_per_km',
    'traffic_lights_per_km', 'bridges_per_km', 'tunnels_per_km',
    'bearing_std', 'turns_per_km', 'mean_circuity', 'edge_length_std', 'edges_per_km'
]

# Procentowe cechy w skali 0-1 (nie wymagają normalizacji)
percentage_features = [
    'pct_high_speed', 'pct_paved', 'pct_unpaved', 'pct_motorway', 'pct_trunk',
    'pct_primary', 'pct_secondary', 'pct_tertiary', 'pct_unclassified',
    'pct_residential', 'pct_lit'
]
features = continuous_features+percentage_features


In [ ]:
import numpy as np

print("Braki danych w kolumnach:")
print(df[features].isna().sum())
print(df[features].isna().sum()[lambda x: x > 0])

# print(df['pct_unpaved'].value_counts())
# # print(df['bridges_per_km'].value_counts())
# columns_to_check = ['tunnels_per_km', 'bridges_per_km', 'pct_motorway', 'pct_trunk', 'pct_lit', 'pct_unpaved']

# # Liczenie zer i innych wartości
# for col in columns_to_check:
#     zeros = (df[col] == 0).sum()
#     non_zeros = (df[col] != 0).sum()
#     print(f"{col}: zeros = {zeros}, non-zeros = {non_zeros}")


Braki danych w kolumnach:
free_flow_time                   0
total_length                     0
mean_speed                       0
speed_std                        0
speed_range                      0
lane_changes_per_km              0
priority_changes_per_km          0
yield_priority_changes_per_km    0
traffic_lights_per_km            0
bridges_per_km                   0
tunnels_per_km                   0
bearing_std                      0
turns_per_km                     0
mean_circuity                    0
edge_length_std                  0
edges_per_km                     0
pct_high_speed                   0
pct_paved                        0
pct_unpaved                      0
pct_motorway                     0
pct_trunk                        0
pct_primary                      0
pct_secondary                    0
pct_tertiary                     0
pct_unclassified                 0
pct_residential                  0
pct_lit                          0
dtype: int64
Series([], dtype

In [ ]:
# Normalizacja cech ciągłych
scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[continuous_features] = scaler.fit_transform(df[continuous_features])
# weighted mean_speed 
weight = 10.0
df_scaled['mean_speed'] = df_scaled['mean_speed'] * weight

X = df_scaled[features].values

print("Dane gotowe do clusteringu:", X.shape)


Dane gotowe do clusteringu: (558, 27)


In [ ]:
from sklearn.cluster import KMeans
import pandas as pd


kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X)
labels = kmeans.labels_

# Dodajemy etykiety do oryginalnego DataFrame dla analizy
df['cluster'] = labels

# Sprawdzenie ile tras w każdym klastrze
print(df['cluster'].value_counts())
df.to_csv("../results/clustered_routes/beynes_routes_clustered.csv", index=False)

cluster
0    321
2    184
1     53
Name: count, dtype: int64


In [40]:

global_speed = df['mean_speed'].mean()
print(global_speed)
# Średnia prędkość w każdym klastrze
cluster_speed = df.groupby('cluster')['mean_speed'].mean()
print(cluster_speed)

# Który klaster najszybszy
fastest_cluster = cluster_speed.idxmax()
print(f"Najszybszy klaster to: {fastest_cluster}")

35.865130772150216
cluster
0    31.397594
1    47.711402
2    40.246799
Name: mean_speed, dtype: float64
Najszybszy klaster to: 1


In [ ]:

global_speed = df['free_flow_time'].mean()
print(global_speed)

cluster_speed = df.groupby('cluster')['free_flow_time'].mean()
print(cluster_speed)

cluster_speed_std = df.groupby('cluster')['free_flow_time'].std()
print(cluster_speed_std)

fastest_cluster = cluster_speed.idxmin()
print(f"Najszybszy(free_flow_time) klaster to: {fastest_cluster}")

3.8021853598750774
cluster
0    2.588955
1    4.035202
2    5.851625
Name: free_flow_time, dtype: float64
cluster
0    1.214708
1    2.700861
2    3.153058
Name: free_flow_time, dtype: float64
Najszybszy(free_flow_time) klaster to: 0


In [ ]:

global_speed = df['total_length'].mean()
print(global_speed)

cluster_speed = df.groupby('cluster')['total_length'].mean()
print(cluster_speed)

fastest_cluster = cluster_speed.idxmin()
print(f"Najkrótszy klaster to: {fastest_cluster}")

2708.537361008298
cluster
0    1576.660972
1    3700.556571
2    4397.424877
Name: total_length, dtype: float64
Najkrótszy klaster to: 0


In [ ]:

global_speed = df['traffic_lights_per_km'].mean()
print(global_speed)
cluster_speed = df.groupby('cluster')['traffic_lights_per_km'].mean()
print(cluster_speed)

fastest_cluster = cluster_speed.idxmin()
print(f"Najmniej swiatel klaster to: {fastest_cluster}")

0.36052672778224626
cluster
0    0.470359
1    0.105177
2    0.242470
Name: traffic_lights_per_km, dtype: float64
Najmniej swiatel klaster to: 1
